In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.stats as stats
import yfinance as yfin

import requests
import xlsxwriter
from datetime import date

import oneMonthRet ##oneMonthRet is a module which reurns the return percentage

# Import the Nifty stocks

In [2]:
###@@@@ ADD .NS to stock names as yahoo uses [StockName].NS
################ NIFTY50 #############################
# fileName='NIFTY50.csv'
# # fileName='MIDCAP50_Yahoo.csv'
# stocks=stockName('NIFTY50.csv')
# column_lst=niftyStocks(stocks)
# # niftyReturn()

def stockName(fileName):
    data=pd.read_csv(fileName)
    stocks=data['Symbol']
    for i in range(0,len(stocks)):
        stocks[i]=stocks[i]+'.NS'
    return stocks

In [3]:
def niftyStocks(stocks):
    yfin
    CMP=[] #current market price
    MC=[] # MarketCap
    TPE=[] #trailing PE
    SECTOR=[]
    dict_nifty_details={} ### this list will contain all the above list
    for name in stocks:
        ticker = yfin.Ticker(name)
        lst=ticker.info.keys()
#         print(ticker.info)
        CMP.append(ticker.info['regularMarketPrice'])
#         MC.append(ticker.info['marketCap'])
#         SECTOR.append(ticker.info['sector'])
        if (list(lst).count('trailingPE')!=0):
            TPE.append(ticker.info['trailingPE'])
        else:
            TPE.append('NaN')
    
    dict_nifty_details={'Name':stocks,'CMP':CMP,'MarketCap':MC,'TrailingPE':TPE,'Sector':SECTOR}
    return dict_nifty_details
        # x= lambda y: TPE.append(tpe) if list(lst).count('trailingPE')!=0 else TPE.append('NaN')

In [4]:
def companyData(name,duration,start,end):
    yfin
    ticker = yfin.Ticker(name)
    if (len(start)!= 0 and len(end)!=0):
        data=ticker.history(start=start,end=end)
    else:
        data=ticker.history(period=duration)
        
    month=data.index.month
    month=month.value_counts().index
    year=data.index.year
    year= year.value_counts().index
    year=year.sort_values(ascending=True)
    return (data,ticker,year,month)

In [5]:
def retForFixedPeriod(data):
    mnth_OP=[]  ##month open price
    mnth_CP=[]  ##month close price
    
    mnth_OP= data['Open'][0]
    mnth_CP= data['Close'][len(data)-1]
    ret=percDiff(mnth_OP,mnth_CP)
    return ret

In [6]:
def percDiff(a,b):
    perc= ((b-a)*100)/a
    return perc

In [7]:
def niftyReturn():    
    month_ret=[] ##monthly return
    three_month_ret=[] ## 3 monthly return
    six_month_ret=[] ## 6 month return
    one_year_ret=[] ## 1 year return
    one_year_percentile=[] # 1 year precentile
    six_month_percentile=[]
    three_month_percentile=[]
    month_percentile=[]
    Avg_Score=[] ## average of all percentiles
    dict_nifty_returns={} ### dictionary containing the data on return percentage
    for name in stocks:
        #OneMonData,ticker,year,month=oneMonthRet.company(name,'','2021-02-01','2021-02-28')
        OneMonData,ticker,year,month=companyData(name,'','2021-02-01','2021-02-28')
        month_ret.append(retForFixedPeriod(OneMonData))
        #month_ret.append(oneMonthRet.OneMonthRet(OneMonData))
        #ThreeMonData,ticker,year,month=oneMonthRet.company(name,'3mo','','')
        ThreeMonData,ticker,year,month=companyData(name,'3mo','','')
        three_month_ret.append(retForFixedPeriod(ThreeMonData))
        #three_month_ret.append(oneMonthRet.OneMonthRet(ThreeMonData))
        #SixMonData,ticker,year,month=oneMonthRet.company(name,'6mo','','')
        SixMonData,ticker,year,month=companyData(name,'6mo','','')
        six_month_ret.append(retForFixedPeriod(SixMonData))
        #six_month_ret.append(oneMonthRet.OneMonthRet(SixMonData))
        #OneYrData,ticker,year,month=oneMonthRet.company(name,'1y','','')
        OneYrData,ticker,year,month=companyData(name,'1y','','')
        one_year_ret.append(retForFixedPeriod(OneYrData))
        #one_year_ret.append(oneMonthRet.OneMonthRet(OneYrData))
    # pd.DataFrame(dict).sort_values(by='1MonthReturn',ascending=False)
    # pd.DataFrame(dict).sort_values(by='MarketCap',ascending=False)
    for i in range(0, len(stocks)):
        x=stats.percentileofscore(month_ret,month_ret[i])/100
        y=stats.percentileofscore(three_month_ret,three_month_ret[i])/100
        z=stats.percentileofscore(six_month_ret,six_month_ret[i])/100
        n=stats.percentileofscore(one_year_ret,one_year_ret[i])/100
        month_percentile.append(x)
        three_month_percentile.append(y)
        six_month_percentile.append(z)
        one_year_percentile.append(n)
        Avg_Score.append((x+y+z+n)/4)
    dict_nifty_returns={'Name':stocks,'1MonthReturn':month_ret,'1MonthPercentile':month_percentile,
      '3MonthReturn':three_month_ret,'3MonthPercentile':three_month_percentile,
     '6MonthReturn':six_month_ret,'6MonthPercentile':six_month_percentile,
      '1YearReturn':one_year_ret,'1YearPercentile':one_year_percentile,'AvgScore':Avg_Score}
    
    return dict_nifty_returns

# NIFTY50
Read the nifty50 file and get the return data

In [8]:
stocks=stockName('NIFTY50.csv')
# stocks=stockName('MIDCAP50_Yahoo.csv')
# stocks=stockName('SMALLCAP50.csv')
#stocks=stockName('MIDCAP150.csv')
dict_nifty_details=niftyStocks(stocks)
dict_nifty_returns=niftyReturn()
# dict1={'Name':stocks,'CMP':CMP,'MarketCap':MC,'TrailingPE':TPE,'1MonthReturn':month_ret,'1MonthPercentile':month_percentile,
#       '3MonthReturn':three_month_ret,'3MonthPercentile':three_month_percentile,
#      '6MonthReturn':six_month_ret,'6MonthPercentile':six_month_percentile,
#       '1YearReturn':one_year_ret,'1YearPercentile':one_year_percentile}
# dict2={'Name':stocks,'CMP':CMP,'MarketCap*10^12':MC,'TrailingPE':TPE,'AvgScore':Avg_Score}
# data_Df=pd.DataFrame(dict2)
# data_Df

C:\Users\PRADEEP\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
df=pd.DataFrame(dict_nifty_details)
df1=pd.DataFrame(dict_nifty_returns)
df1=df1[['Name','AvgScore']]
df2=pd.merge(df,df1,how='inner',on='Name')
df2.sort_values(by='AvgScore',ascending=False)[:50]

NameError: name 'dict_nifty_details' is not defined

In [ ]:
# data_Df['']
data_Df.sort_values(by='AvgScore',ascending=False)

# NIFTY MIDCAP50
Read the nifty50 file and get the return data

In [ ]:
stocks=stockName('MIDCAP50_Yahoo.csv')
niftyStocks(stocks)
niftyReturn()
# niftyReturn()

In [ ]:
dict1_midCap={'Name':stocks,'CMP':CMP,'MarketCap':MC,'TrailingPE':TPE,'1MonthReturn':month_ret,'1MonthPercentile':month_percentile,
      '3MonthReturn':three_month_ret,'3MonthPercentile':three_month_percentile,
     '6MonthReturn':six_month_ret,'6MonthPercentile':six_month_percentile,
      '1YearReturn':one_year_ret,'1YearPercentile':one_year_percentile}
dict2_midCap={'Name':stocks,'CMP':CMP,'MarketCap*10^12':MC,'TrailingPE':TPE,'AvgScore':Avg_Score}
data_Df=pd.DataFrame(dict2_midCap)
data_Df

In [ ]:
import datetime
start = datetime.datetime(2012,5,31)
end = datetime.datetime(2018,3,1)
stocks1=['UPL.NS','WIPRO.NS']
data1 = yfin.download(stocks, start=start, end=end)

# No.of Days Nifty Ended positive and Negative
To calculate no.of positve and negative days in a annual year

# P/E Ratio of Nifty

In [ ]:
import googlefinance as gfin

In [ ]:
import json

import requests

rsp = requests.get('https://finance.google.com/finance?q=AAPL&output=json')
fin_data = json.loads(rsp.content[6:-2].decode('unicode_escape'))


In [ ]:
rsp